In [2]:
import requests
from bs4 import BeautifulSoup
import time
import csv
from urllib.parse import quote # URL-д кирилл үсгийг кодлоход

BASE_SEARCH_URL = "https://bolor-toli.com/result/"
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
OUTPUT_FILENAME = 'bolor_toli_search_results.csv'
REQUEST_TIMEOUT = 20
RETRY_COUNT = 2
RETRY_DELAY = 10 # Хайлтын системд болгоомжтой хандах үүднээс илүү удаан хүлээх
INTER_REQUEST_DELAY = 3 # Үг хоорондын үндсэн хүлээлэг (секунд)

# --- Таны хайх үгсийн жагсаалт ---
# Энэ жагсаалтыг өөрийнхөөрөө дүүргэнэ үү
WORDS_TO_SEARCH = [
    "нэр",
    "юу",
    "баатар",
    "ном",
    "гэр",
    "аав",
    "ээж",
    "хүн",
    "амьдрал",
    "хайр"
    # ... нэмж үгсээ оруулна уу ...
]
# ---------------------------------

def make_request_with_retry(url, stream=False):
    for attempt in range(RETRY_COUNT):
        try:
            print(f"  Хүсэлт илгээж байна: {url}")
            response = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT, stream=stream)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            print(f"  Хүсэлт ({attempt+1}/{RETRY_COUNT}) алдаатай: {url} - {e}")
            if attempt < RETRY_COUNT - 1:
                print(f"    Дахин оролдоно {RETRY_DELAY} секундын дараа...")
                time.sleep(RETRY_DELAY * (attempt + 1))
            else:
                print(f"  {RETRY_COUNT} удаагийн оролдлогын дараа {url}-г татаж чадсангүй.")
                return None

def extract_data_from_search_result_page(html_content, searched_word_cyrillic):
    """
    Хайлтын үр дүнгийн хуудаснаас кирилл болон монгол бичгийг гаргаж авна.
    Энэ функц нь хайлтын үр дүнгийн хуудасны HTML бүтцээс хамаарна.
    Таны өгсөн HTML (<span id="resultWord" ...>) бүтэцтэй төстэй байх магадлалтай.
    """
    try:
        soup = BeautifulSoup(html_content, 'lxml') # lxml суулгасан гэж үзвэл
        # soup = BeautifulSoup(html_content, 'html.parser') # lxml байхгүй бол

        # Хайлтын үр дүнгийн хуудасны бүтцийг шалгах хэрэгтэй.
        # Энэ бол таамаглал. Яг ямар элементэд үгс байгааг Developer Tools-оор шалгаарай.
        
        # Жишээ нь, хайлтын үр дүнгийн жагсаалтад олон үг байж болно.
        # Бид зөвхөн хайсан үгтэйгээ аль болох ойролцоо эсвэл эхний үр дүнг авахыг оролдоно.

        # Боломжит хувилбар 1: Үг бүр тусдаа "card" эсвэл "item" дотор байх
        # result_items = soup.find_all('div', class_='result-item') # Эсвэл өөр класс
        
        # Боломжит хувилбар 2: Таны үзүүлсэн HTML бүтэц шууд ашиглагдах
        # Эхний resultWord-г олохыг оролдъё
        
        # Энэ хэсгийг хайлтын үр дүнгийн хуудасны бодит бүтцэд тааруулах шаардлагатай!
        # Доорх нь таны өмнө үзүүлсэн HTML бүтэц дээр тулгуурласан таамаглал юм.

        # Ихэвчлэн хайлтын үр дүнд олон үг гарч ирдэг.
        # Бид хайсан үгтэйгээ яг таарч буй (эсвэл хамгийн эхний) үр дүнг олохыг хичээнэ.
        
        # Тухайн үгийг агуулсан хэсгийг олох (энэ нь маш ерөнхий)
        # Жишээ нь, үг бүр "entry" гэсэн div-д байж болно
        entries = soup.find_all(lambda tag: tag.name == 'a' and tag.has_attr('href') and '/word/' in tag['href'] and tag.find('span', class_='translate-items'))
        # Дээрх нь /word/ хуудас руу холбосон, дотроо translate-items класс_тай span агуулсан 'a' тагийг хайна.
        # Энэ нь маш их таамаг бөгөөд сайтын бүтцээс хамаарна.

        found_word_data = None

        if not entries: # Хэрэв дээрх аргаар олоогүй бол таны өгсөн span-г хайх
            # Энэ нь зөвхөн ганц үр дүн буцаах үед ажиллаж магадгүй.
            cyrillic_span_container = soup.find('span', id='resultWord', class_='translate-items')
            mongolian_span = soup.find('span', id='resultWord', class_='sub-variant mScript')

            if cyrillic_span_container and mongolian_span:
                cyrillic_inner_span = cyrillic_span_container.find('span')
                if cyrillic_inner_span:
                    # b тагуудыг нэгтгэх эсвэл зүгээр текст авах
                    cyrillic_text = cyrillic_inner_span.get_text(separator=' ', strip=True)
                    
                    if mongolian_span.sup: # <sup>mb</sup> хэсгийг арилгах
                        mongolian_span.sup.decompose()
                    mongolian_text = mongolian_span.get_text(strip=True)

                    if cyrillic_text.lower() == searched_word_cyrillic.lower(): # Хайсан үгтэй таарч байвал
                        found_word_data = {'cyrillic': cyrillic_text, 'mongolian_script': mongolian_text}

        else: # Хэрэв 'entries' олдсон бол эхнийхийг нь эсвэл таарсныг нь авахыг оролдох
            for entry in entries:
                cyrillic_span = entry.find('span', class_='translate-items')
                mongolian_span = entry.find_next_sibling('span', class_='mScript') # Эсвэл өөр хамаарал
                
                # Заримдаа монгол бичиг нь a тагийн дараагийн sibling биш, харин өөр газарт байж болно.
                # Хэрэв mScript нь a таг доторх өөр элемент бол:
                if not mongolian_span:
                     # Жишээ нь, хэрэв монгол бичиг нь a таг доторх өөр span байвал:
                     possible_mongol_container = entry.find(lambda t: t.name == 'span' and 'mScript' in t.get('class', []))
                     if possible_mongol_container:
                         mongolian_span = possible_mongol_container


                if cyrillic_span and mongolian_span:
                    # Кирилл үгийг span доторх span-аас эсвэл шууд авах
                    inner_cyrillic_span = cyrillic_span.find('span')
                    if inner_cyrillic_span:
                         # b тагууд байвал нэгтгэж авах
                         cyrillic_text = inner_cyrillic_span.get_text(separator=' ', strip=True)
                    else:
                         cyrillic_text = cyrillic_span.get_text(strip=True)
                    
                    # Монгол бичиг
                    if mongolian_span.sup:
                        mongolian_span.sup.decompose()
                    mongolian_text = mongolian_span.get_text(strip=True)

                    # Хайсан үгтэйгээ таарч байгаа эсэхийг шалгах (эсвэл эхнийхийг авах)
                    if cyrillic_text.lower() == searched_word_cyrillic.lower():
                        found_word_data = {'cyrillic': cyrillic_text, 'mongolian_script': mongolian_text}
                        break # Олдсон тул давталтаас гарах
                
            if not found_word_data and entries: # Хэрэв яг таарсан нь олдоогүй ч үр дүн байвал эхнийхийг авч үзэх
                print(f"  '{searched_word_cyrillic}' гэсэн үгтэй яг таарсан үр дүн олдсонгүй. Эхний үр дүнг авч байна (хэрэв байгаа бол).")
                first_entry = entries[0]
                cyrillic_span = first_entry.find('span', class_='translate-items')
                mongolian_span = first_entry.find_next_sibling('span', class_='mScript')
                # ... (дээрхтэй ижил логикоор эхний үр дүнг задлах) ...
                # Энэ хэсгийг гүйцээх шаардлагатай. Сайтын HTML бүтцээс хамаарна.


        if found_word_data:
            return found_word_data
        else:
            print(f"  '{searched_word_cyrillic}' гэсэн үгэнд тохирох мэдээлэл олдсонгүй.")
            return None

    except Exception as e:
        print(f"  Хайлтын үр дүн задлахад алдаа: {e}")
        import traceback
        print(traceback.format_exc()) # Алдааны дэлгэрэнгүйг харах
        return None

# --- Үндсэн логик ---
all_found_words_data = []

print(f"{len(WORDS_TO_SEARCH)} үгээр хайлт хийж эхэллээ...")

for i, word_to_search in enumerate(WORDS_TO_SEARCH):
    print(f"\nХайж байна ({i+1}/{len(WORDS_TO_SEARCH)}): '{word_to_search}'")
    
    # URL-д зориулж үгийг encode хийх
    encoded_word = quote(word_to_search)
    search_url = f"{BASE_SEARCH_URL}?word={encoded_word}&direction=1" # direction=1 нь Монгол-Монгол гэж үзлээ
    
    response = make_request_with_retry(search_url)
    
    if response:
        word_data = extract_data_from_search_result_page(response.content, word_to_search)
        if word_data:
            all_found_words_data.append(word_data)
            print(f"  Олдлоо: {word_data}")
        
        # Хүсэлт хооронд хүлээх (маш чухал!)
        actual_delay = INTER_REQUEST_DELAY + (i % 5) * 0.5 # Бага зэрэг санамсаргүй байдал нэмэх
        print(f"  Дараагийн хүсэлт хүртэл {actual_delay:.1f} секунд хүлээж байна...")
        time.sleep(actual_delay)
    else:
        print(f"  '{word_to_search}' гэсэн үгийн хайлтын хуудсыг татаж чадсангүй.")
        # Алдаа гарсан ч дараагийн үг рүү орохын өмнө хүлээх
        time.sleep(RETRY_DELAY)


# Үр дүнг CSV файлд хадгалах
if all_found_words_data:
    print(f"\nНийт {len(all_found_words_data)} үгийн мэдээлэл олж, '{OUTPUT_FILENAME}' файлд хадгаллаа.")
    with open(OUTPUT_FILENAME, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['cyrillic', 'mongolian_script']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(all_found_words_data)
    print(f"Үр дүнг '{OUTPUT_FILENAME}' файлд амжилттай хадгаллаа.")
else:
    print("\nХайлтын үр дүнд ямар ч үгийн мэдээлэл цуглуулж чадсангүй.")

10 үгээр хайлт хийж эхэллээ...

Хайж байна (1/10): 'нэр'
  Хүсэлт илгээж байна: https://bolor-toli.com/result/?word=%D0%BD%D1%8D%D1%80&direction=1
  'нэр' гэсэн үгэнд тохирох мэдээлэл олдсонгүй.
  Дараагийн хүсэлт хүртэл 3.0 секунд хүлээж байна...

Хайж байна (2/10): 'юу'
  Хүсэлт илгээж байна: https://bolor-toli.com/result/?word=%D1%8E%D1%83&direction=1
  'юу' гэсэн үгэнд тохирох мэдээлэл олдсонгүй.
  Дараагийн хүсэлт хүртэл 3.5 секунд хүлээж байна...

Хайж байна (3/10): 'баатар'
  Хүсэлт илгээж байна: https://bolor-toli.com/result/?word=%D0%B1%D0%B0%D0%B0%D1%82%D0%B0%D1%80&direction=1
  'баатар' гэсэн үгэнд тохирох мэдээлэл олдсонгүй.
  Дараагийн хүсэлт хүртэл 4.0 секунд хүлээж байна...

Хайж байна (4/10): 'ном'
  Хүсэлт илгээж байна: https://bolor-toli.com/result/?word=%D0%BD%D0%BE%D0%BC&direction=1
  'ном' гэсэн үгэнд тохирох мэдээлэл олдсонгүй.
  Дараагийн хүсэлт хүртэл 4.5 секунд хүлээж байна...


KeyboardInterrupt: 